In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [3]:
csv_path = "D:\\workspace\\DeepLearning_codes\\AlphaReal\\200107_SO_test.csv"
df = pd.read_csv(csv_path)
print(df.head())

  Date Time     MM  Price_index  Permits     JS  JS_Ratio  Mort_Int
0  2003. 11 -0.649       73.943     2137 -0.990      53.6      6.04
1  2003. 12 -0.327       74.269    17337 -0.400      53.3      6.28
2  2004. 01  0.656       74.728     1411  0.402      53.2      6.29
3  2004. 02  0.489       75.055     1218  0.600      53.3      6.15
4  2004. 03  0.648       75.775     9620  0.000      53.7      5.98


In [4]:
csv_path = "D:\\workspace\\DeepLearning_codes\\AlphaReal\\200107_BS_test.csv"
df2 = pd.read_csv(csv_path)
print(df2.head())

  Date Time     MM  Price_index  Permits     JS  JS_Ratio  Mort_Int
0  2003. 11 -0.836       73.744     5997 -1.049      69.4      6.04
1  2003. 12 -0.169       73.744     4170 -0.530      69.3      6.28
2  2004. 01  0.000       74.075     2337  0.000      69.1      6.29
3  2004. 02  0.169       74.405     1497  0.355      69.2      6.15
4  2004. 03  0.169       75.199      376  0.354      69.3      5.98


In [5]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

In [6]:
TRAIN_SPLIT = 100
tf.random.set_seed(13)

In [7]:
def build_train_val_data(df):
    uni_data = df['MM']
    uni_data.index = df['Date Time']
    #uni_data.head()

    #uni_data.plot(subplots=True)
    #uni_data.describe()

    uni_data = uni_data.values
    uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
    uni_train_std = uni_data[:TRAIN_SPLIT].std()

    uni_data = (uni_data-uni_train_mean)/uni_train_std

    ## 24 month history prediction
    univariate_past_history = 24
    univariate_future_target = 0

    x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT, 
                                                univariate_past_history,
                                                univariate_future_target)                                           
    x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None, 
                                                univariate_past_history, 
                                                univariate_future_target)
                                  
    return x_train_uni, y_train_uni, x_val_uni, y_val_uni

In [8]:
x_train_uni, y_train_uni, x_val_uni, y_val_uni = build_train_val_data(df)
x_train_uni2, y_train_uni2, x_val_uni2, y_val_uni2 = build_train_val_data(df2)

In [9]:
x_train_uni.shape

(76, 24, 1)

In [12]:
x_train_uni2.shape

(76, 24, 1)

In [13]:
y_train_uni.shape

(76,)